In [1]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azure-mgmt-resource 18.0.0 (c:\users\souvik\appdata\roaming\python\python36\site-packages), Requirement.parse('azure-mgmt-resource<15.0.0,>=1.2.1'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azure-mgmt-resource 18.0.0 (c:\users\souvik\appdata\roaming\python\python36\site-packages), Requirement.parse('azure-mgmt-resource<15.0.0,>=1.2.1'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azure-mgmt-resource 18.0.0 (c:\users\souvik\appdata\roaming\python\python36\site-packages), Requirement.parse('azure-mgmt-resource<15.0.0,>=1.2.1'), {'azureml-core'}).
Failure while loa

In [2]:
azureml.core.VERSION

'1.31.0'

In [3]:
from azureml.core import Workspace, Experiment

ws= Workspace.from_config("./config_paid.json")
#ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

#run = exp.start_logging()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace name: Udacity_learn
Azure region: southeastasia
Subscription id: c0a4a4eb-e8e6-4dfe-900a-927836f229e2
Resource group: MLOps


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
amlcompute_cluster_name = "cpu-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True)


Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import Environment

sklearn_env = Environment.get(workspace=ws, name='AzureML-sklearn-0.24-ubuntu18.04-py37-cuda11-gpu')

# Specify parameter sampler
ps= RandomParameterSampling({
    "C": uniform(0.8, 1.2),
    "max_iter": choice(75, 100, 120, 130)
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval= 4, slack_factor = 0.1)


# Create a SKLearn estimator for use with train.py
sk_est = ScriptRunConfig(
			source_directory= "./",
			compute_target= aml_compute,
			script= "train.py",
			environment= sklearn_env
			)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(ps,
										run_config= sk_est,
										primary_metric_name= "Accuracy",
										primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
										max_total_runs= 100,
										policy= policy
										)

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import Environment
import joblib
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
import pandas as pd
from train import clean_data
from azureml.train.automl import AutoMLConfig

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

exp_run_obj= exp.submit(hyperdrive_config)

In [7]:
RunDetails(exp_run_obj).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
import joblib

# Get your best run and save the model from that run.

if not os.path.exists("./outputs"):
	os.makedirs("./outputs")

best_run= exp_run_obj.get_best_run_by_primary_metric()
best_run_results= best_run.get_metrics()
print("Best Accuracy= {}".format(best_run_results['Accuracy']))


Best Accuracy= 0.9133535660091047


In [10]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_6bea033c26e1e2ca5c8d6bc56bc78b2dc5efa5f6cd546ea3084df931901d0dbd_d.txt',
 'azureml-logs/65_job_prep-tvmps_6bea033c26e1e2ca5c8d6bc56bc78b2dc5efa5f6cd546ea3084df931901d0dbd_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_6bea033c26e1e2ca5c8d6bc56bc78b2dc5efa5f6cd546ea3084df931901d0dbd_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/94_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/auto_ml.pkl',
 'outputs/auto_ml_conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/model-C=0.8092154598113208--max_iter=130.joblib']

In [11]:
best_run.download_file(best_run.get_file_names()[-1], output_file_path = "./outputs")

In [13]:
best_run.register_model(model_name= 'hyperdrive_best', 
                        model_path= "./outputs/model-C=0.8092154598113208--max_iter=130.joblib")

Model(workspace=Workspace.create(name='udacity_pipeline', subscription_id='8bb0c18e-6b4b-4695-bd30-aff7b7ff6815', resource_group='Udacity'), name=hyperdrive_best, id=hyperdrive_best:3, version=3, tags={}, properties={})

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
import pandas as pd

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
az_dataset= Dataset.Tabular.from_delimited_files(path= data_url)
print(az_dataset.to_pandas_dataframe().head())

   age          job  marital    education  default housing loan    contact  \
0   57   technician  married  high.school       no      no  yes   cellular   
1   55      unknown  married      unknown  unknown     yes   no  telephone   
2   33  blue-collar  married     basic.9y       no      no   no   cellular   
3   36       admin.  married  high.school       no      no   no  telephone   
4   27    housemaid  married  high.school       no     yes   no   cellular   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         1      failure         -1.8   
1   may         thu  ...         2    999         0  nonexistent          1.1   
2   may         fri  ...         1    999         1      failure         -1.8   
3   jun         fri  ...         4    999         0  nonexistent          1.4   
4   jul         fri  ...         2    999         0  nonexistent          1.4   

   cons.price.idx  cons.conf.idx  euribor3m 

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
X_df, y_df= clean_data(az_dataset)
X_df["y"]= y_df

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
default_storage= ws.get_default_datastore()
print(default_storage)
automl_data= Dataset.Tabular.register_pandas_dataframe(X_df, default_storage, name= "X_df")
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data= automl_data,
    label_column_name='y',
    compute_target= aml_compute,
    max_cores_per_iteration= 4,
    max_concurrent_iterations=4,
    n_cross_validations = 4
    )

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-99c82bfb-eb50-4353-9562-87fbcd7fe827",
  "account_name": "udacitylearn0250132790",
  "protocol": "https",
  "endpoint": "core.windows.net"
}
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/09535406-d70a-44b7-810f-c3ed64e66838/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [9]:
# Submit your automl run
exp_automl = Experiment(workspace=ws, name="Automl-Experiment")
#exp.start_logging()
automl_run = exp_automl.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Automl-Experiment,AutoML_3a7f9f7e-14d4-4877-8cea-e382fc398fc0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
exp_automl = Experiment(workspace=ws, name="Automl-Experiment")

In [15]:
from azureml.train.automl.run import AutoMLRun
automl_run = AutoMLRun(exp_automl, "AutoML_3a7f9f7e-14d4-4877-8cea-e382fc398fc0")

In [16]:
print(automl_run)

Run(Experiment: Automl-Experiment,
Id: AutoML_3a7f9f7e-14d4-4877-8cea-e382fc398fc0,
Type: automl,
Status: Completed)


In [18]:
best_run, fitted_model= automl_run.get_output()
print(best_run)
print("\n\n")
print(fitted_model)

Run(Experiment: Automl-Experiment,
Id: AutoML_3a7f9f7e-14d4-4877-8cea-e382fc398fc0_75,
Type: azureml.scriptrun,
Status: Completed)



Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='F:\\ML_Projects\\AzureMLPipeline')),
                (...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=None, reg_alpha=0, reg_lambda=0.9473684210526315, subsample=1))], verbose=False))], flatten_transform=None, weights=[0.14285714285714285, 0.14285714285714285, 0.21428571428571427, 0.21428571428571427, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]))],
         verbose=False)


In [19]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_9cf6b419226930355c722a0b00b01e9a40563f24a27fabafe8e56e81daab29cc_d.txt',
 'azureml-logs/65_job_prep-tvmps_9cf6b419226930355c722a0b00b01e9a40563f24a27fabafe8e56e81daab29cc_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_9cf6b419226930355c722a0b00b01e9a40563f24a27fabafe8e56e81daab29cc_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'explanation/1dfd2574/classes.interpret.json',
 'explanation/1dfd2574/eval_data_viz.interpret.json',
 'explanation/1dfd2574/expected_values.interpret.json',
 'explanation/1dfd2574/features.interpret.json',
 'explanation/1dfd2574/global_names/0.interpret.json',
 'explanation/1dfd2574/global_rank/0.interpret.json',
 'explanation/1dfd2574/global_values/0.interpret.json',
 'explanation/1dfd2574/local_importance_values.interpret.json',
 'explanation/1dfd2574/per_class_names/0.interpret.json',
 'explanati

In [20]:
best_run.download_file('outputs/model.pkl', "outputs/auto_ml.pkl")
best_run.download_file('outputs/conda_env_v_1_0_0.yml', "outputs/auto_ml_conda_env_v_1_0_0.yml")
best_run.download_file('outputs/env_dependencies.json', "outputs/env_dependencies.json")

In [21]:
best_run.register_model(model_name= 'hyperdrive_best', 
                        model_path= "./outputs/model.pkl")

Model(workspace=Workspace.create(name='Udacity_learn', subscription_id='c0a4a4eb-e8e6-4dfe-900a-927836f229e2', resource_group='MLOps'), name=hyperdrive_best, id=hyperdrive_best:1, version=1, tags={}, properties={})

In [22]:
aml_compute.delete()

NameError: name 'aml_compute' is not defined